In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
data_name = '' # quantifier or numbers
data = pd.read_csv(f'./Data/data_{data_name}.csv')

In [ ]:
# mdeberta: 'MoritzLaurer/mDeBERTa-v3-base-mnli-xnli'
# bertin: 'bertin-project/bertin-base-xnli-es'
# roberta: 'joeddav/xlm-roberta-large-xnli'

name = ''
model_ckp = ''

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckp)
model = AutoModelForSequenceClassification.from_pretrained(model_ckp)

In [ ]:
def inference_task(premise, hypothesis):
    # premise: target sentence (unos or algunos)
    # hypothesis: context sentence (whole or subset)
    # tokenize and get scores
    input = tokenizer(premise, hypothesis, return_tensors='pt')
    output = model(input['input_ids'])
    prediction = torch.softmax(output["logits"][0], -1).tolist()
    
    # map scores to labels and retrieve label name
    labels = list(model.config.id2label.values())
    results = {key: value for key, value in zip(labels, prediction)}
    label = max(results, key=results.get)

    return label

In [ ]:
data[name] = [inference_task(*item) for item in tuple(zip(data['premise'], data['hypothesis']))]

In [ ]:
data.to_csv(f'./Results/results_{data_name}', index=False)